<a href="https://colab.research.google.com/github/RishiLal/CV-FirstHtmlTemplate/blob/main/MLB_Fine_tuning_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader
import os

!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip
#Loading the data
import os
import zipfile

local_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp') # Unzipping the data
zip_ref.close()

base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Define paths to the dataset
data_dir = "/tmp/cats_and_dogs_filtered"
train_dir = os.path.join(data_dir, 'train')
validation_dir = os.path.join(data_dir, 'validation')

# Set up data transformations for training and validation
data_transforms = {
    'train': transforms.Compose([#Performing actions to prepare the images for training
        transforms.RandomResizedCrop(150),#Cropping the image to 150 pixels randomly
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'validation': transforms.Compose([#performing the transformations to all images for validation
        transforms.Resize(160),
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

# Load datasets
train_dataset = datasets.ImageFolder(train_dir, data_transforms['train'])
val_dataset = datasets.ImageFolder(validation_dir, data_transforms['validation'])
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# Loading in the pretrained resnet50 model and modifying it
model = models.resnet50(pretrained=True)
for param in model.parameters():
    param.requires_grad = False

# Replace the final fully connected layer to output 1 class for binary classification
num_features = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_features, 1),
    nn.Sigmoid()
) # finetuning the layer to work for binary classification

# Set device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss and optimizer
criterion = nn.BCELoss()  # Binary Cross Entropy Loss for binary classification
optimizer = optim.Adam(model.fc.parameters(), lr=0.001)#Using the adam optimizer

# Training loop
def train_model(model, criterion, optimizer, num_epochs=10):
    for epoch in range(num_epochs):#running a loop through the data for 10 epochs
        print(f"Epoch {epoch+1}/{num_epochs}")

        # Training phase
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader: # for loop
            inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)

            optimizer.zero_grad()#Resets the previous gradient.
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()#Backpropogation
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)

        epoch_loss = running_loss / len(train_loader.dataset)
        print(f"Training Loss: {epoch_loss:.4f}")

        model.eval()
        running_loss = 0.0
        correct = 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device).float().unsqueeze(1)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)

                preds = outputs > 0.5 #Predicts the classification of the image based on the probability of it being more than 50%
                correct += torch.sum(preds == labels)

        epoch_loss = running_loss / len(val_loader.dataset)
        accuracy = correct.double() / len(val_loader.dataset)
        print(f"Validation Loss: {epoch_loss:.4f} Accuracy: {accuracy:.4f}")

train_model(model, criterion, optimizer)


--2024-11-01 21:28:57--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.197.207, 74.125.135.207, 142.250.99.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.197.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   176MB/s    in 0.4s    

2024-11-01 21:28:57 (176 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]

Epoch 1/10
Training Loss: 0.3137
Validation Loss: 0.1219 Accuracy: 0.9680
Epoch 2/10
Training Loss: 0.1955
Validation Loss: 0.0953 Accuracy: 0.9720
Epoch 3/10
Training Loss: 0.1584
Validation Loss: 0.0760 Accuracy: 0.9770
Epoch 4/10
Training Loss: 0.1555
Validation Loss: 0.0661 Accuracy: 0.9790
Epoch 5/10
Training Loss: 0.1624
Validation Loss: 0.0667 Accuracy: 0.9740
Epoch 6/10
Tra

In [6]:
import torchvision.transforms as transforms
from PIL import Image

# Function to load and preprocess the image
def load_image(image_path):
    image = Image.open(image_path).convert("RGB")
    preprocess = transforms.Compose([
        transforms.Resize(160),  # Resizing the image to 160 by 160 pixels
        transforms.CenterCrop(150),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
    image = preprocess(image).unsqueeze(0)  # Adds the batch dimension for the image
    return image

# Function to predict if the image is a cat or dog
def predict(image_path, model):
    model.eval()  # Set the model to evaluation mode
    image = load_image(image_path)  # Load and preprocess the image
    image = image.to(device)  # Move the image to the same device as the model

    with torch.no_grad():
        output = model(image)
        prediction = (output > 0.5).int()  # Apply threshold to get class prediction

    if prediction.item() == 1: #If statement
        return "Dog"
    else:
        return "Cat"

# Example usage
image1_path =  '/content/Dog_img.jpeg' # Downloaded an image from the internet, the image is of a dog
image2_path =  '/content/Cat_img.jpg' # Downloaded an image from the internet, the image is of a cat

image1_prediction = predict(image1_path, model)# using the model to predict and run for myself.
image2_prediction = predict(image2_path, model)

print(f"The model predicts that the image '{image1_path}' is a {image1_prediction}.")
print(f"The model predicts that the image '{image2_path}' is a {image2_prediction}.")


The model predicts that the image '/content/Dog_img.jpeg' is a Dog.
The model predicts that the image '/content/Cat_img.jpg' is a Cat.
